In [1]:
experiment_name = "emotions_exp"

In [2]:
initial_prompt = '''
Identify the emotion behind the words written.

The possible emotions are: sadness, anger, love, surprise, fear, happy.

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "emotion": "The identified emotion"
}
Ensure that "chain_of_thought" contains your detailed analysis, and "emotion" is strictly one of the six possible emotions listed above.
'''

In [3]:
# Output format prompt
output_format_prompt = '''
Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "emotion": "The identified emotion" - the ONLY possible emotions are: sadness, anger, love, surprise, fear, happy.
}
Ensure that "chain_of_thought" contains your detailed analysis, and "emotion" is strictly one of the six possible emotions listed above.
'''

In [4]:
# Define output schema
output_schema = {
    'key_to_extract': 'emotion',
    'value_mapping': {
        'sadness': 'sadness',
        'anger': 'anger',
        'love': 'love',
        'surprise': 'surprise',
        'fear': 'fear',
        'happy': 'happy'
    },
    'regex_pattern': r'"emotion":\s*"(\w+)"',
    #
    'chain_of_thought_key': 'chain_of_thought',  
    'chain_of_thought_regex': r'"chain_of_thought":\s*"(.*?)"',
    #
    'use_json_mode': True,
}

In [5]:
# Set number of optimization iterations
iterations = 3

In [6]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [7]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "emotions.csv"
text_column = "Text"
target_column = "Emotion"
sample_size = 3

------------------------------------------------------------------------------------------

In [8]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [9]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=[text_column, target_column])
eval_data.columns = ['text', 'label']
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=round(sample_size/2), random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (12, 2)
                                                text     label
0      i remember last summer feeling so overwhelmed  surprise
1  i feel like i have missed out on every single ...   sadness
2              i feel greedy with my self as of late     anger
3                        i feel useless return false   sadness
4  Never before had Bodmin seen so many faces  an...     happy


In [10]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt = initial_prompt,
    eval_data = eval_data,
    iterations =iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    eval_temperature=0.7,
    optim_provider=optim_provider,
    optim_model=optim_model,
    optim_temperature=0,
    use_cache=True,
    output_format_prompt = output_format_prompt,
    output_schema=output_schema,
    fp_comments = "",
    fn_comments = "",
    tp_comments = "",
    invalid_comments="",
    validation_comments="",
    experiment_name = experiment_name,
    skip_prompt_validation = True,
)

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Evaluation temperature: 0.7
Selected optimization provider: ollama
Selected optimization model: llama3.1
Optimization temperature: 0
Estimated token usage: 20472
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│                                                                                                                 │
│ Identify the emotion behind the words written.                                                                  │
│                                                                                                                 │
│ The possible emotions are: sadness, anger, love, surprise, fear, happy.                                         │
│                                                                                                                 │
│ Provide your response as a JSON dictionary with the following structure:                                        │
│ {                                                                                                               │
│     "chain_of_thought": "Your step-by-step reasoning here",                                                     │
│     "emotion": "The identified emotion"                                                                         │
│ }                                                                                                               │
│ Ensure that "chain_of_thought" contains your detailed analysis, and "emotion" is strictly one of the six        │
│ possible emotions listed above.                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

-----------------------------------
Processing text 1/12 .....
Using cached output for text 1/12
Prediction 1/12: sadness | Ground Truth: surprise ❌ (FN)
-----------------------------------
Processing text 2/12 .....
Using cached output for text 2/12
Prediction 2/12: happy | Ground Truth: sadness ❌ (FN)
-----------------------------------
Processing text 3/12 .....
Using cached output for text 3/12
Prediction 3/12: sadness | Ground Truth: anger ❌ (FN)
-----------------------------------
Processing text 4/12 .....
Using cached output for text 4/12
Prediction 4/12: sadness | Ground Truth: sadness ✅ (TN)
-----------------------------------
Processing text 5/12 .....
Using cached output for text 5/12
Prediction 5/12: happy | Ground Truth: happy ✅ (TN)
-----------------------------------
Processing text 6/12 .....
Using cached output for text 6/12
Prediction 6/12: fear | Ground Truth: fear ✅ (TN)
-----------------------------------
Processing text 7/12 .....
Using cached output for text 7/1

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].